In [1]:
import pandas as pd
import numpy as np

In [2]:
ttino = pd.read_csv('tarantino.csv')
ttino.head()

,movie,type,word,minutes_in
0,Reservoir Dogs,word,dick,0.40
1,Reservoir Dogs,word,dicks,0.43
2,Reservoir Dogs,word,fucked,0.55
3,Reservoir Dogs,word,fucking,0.61
4,Reservoir Dogs,word,bullshit,0.61


Проверим, есть ли пропущенные значения.

In [10]:
ttino.isnull().sum()

movie           0
type            0
word          190
minutes_in      0
dtype: int64

Найдено 190 пропущенных значений в столбце word.

Посмотрим, какие значения присутствуют в столбце type, чтобы определить, каким type соответствуют пропущенные значения из столбца word.

In [15]:
ttino['type'].value_counts()

type
word     1704
death     190
Name: count, dtype: int64

В столбце type присутствуют только два уникальных значения: word в 1704 строчках и death в 190. 

Так как количество строк с type death равно количеству строк с пропущенным значением в столбце word, то можно сделать предположение, что это одни и те же строки, то есть чт именно в строчках с type death не прописано значение в столбце word. Проверим это:

In [29]:
ttino[ttino['type'] == 'death'].isnull().sum()

movie           0
type            0
word          190
minutes_in      0
dtype: int64

Получили, что в 190 строчках с type death присутсвует пропущенное значение в столбце word. Таким образом, это значит, что пропущенные значения есть только в type death, а в свою очередь, в type word такие значения отсутсвуют. Следовательно, мы знаем каждое конкретное слово.

Итак, перейдем к первому пункту задания и определим число летальных исходов, проклятий и их соотношение в каждом отдельном фильме:

In [31]:
agg_df = ttino.groupby(['movie', 'type']).size().unstack(fill_value=0)
agg_df['total'] = agg_df.sum(axis=1)

agg_df['death_percentage'] = round(100.0 * agg_df['death'] / agg_df['total'],1)
agg_df['word_percentage'] = round(100.0 * agg_df['word'] / agg_df['total'],1)

agg_df

type,death,word,total,death_percentage,word_percentage
movie,,,,,
Django Unchained,47,262,309,15.2,84.8
Inglorious Basterds,48,58,106,45.3,54.7
Jackie Brown,4,368,372,1.1,98.9
Kill Bill: Vol. 1,63,57,120,52.5,47.5
Kill Bill: Vol. 2,11,69,80,13.8,86.2
Pulp Fiction,7,469,476,1.5,98.5
Reservoir Dogs,10,421,431,2.3,97.7


Далее, определим частоту употребления конкретных проклятий, то есть выведем, сколько раз сказано каждое проклятие и сколько это составляет в процентах от общего числа проклятий:

In [8]:
agg_df_words = ttino[ttino['type'] == 'word'].groupby('word', as_index = False).agg(amount = ('word', 'count'))


agg_df_words['total'] = agg_df_words['amount'].sum()
agg_df_words['word_percentage'] = round(100.0 *agg_df_words['amount'] / agg_df_words['total'],1)
agg_df_words = agg_df_words[['word', 'amount', 'word_percentage']]

agg_df_words

,word,amount,word_percentage
0,ass,140,8.2
1,asses,4,0.2
2,asshead,1,0.1
3,asshole,10,0.6
4,assholes,5,0.3
5,bastard,10,0.6
6,bastards,1,0.1
7,bitch,64,3.8
8,bitches,9,0.5
9,bullshit,15,0.9


Затем, определим распределение времени между проклятиями, то есть медиану времени, а также квантиль 0.25, 0.75:

In [39]:
agg_df_q = ttino[ttino['type'] == 'word'].groupby('word', as_index = False).agg(q_25_minutes = ('minutes_in', lambda x: x.quantile(0.25)),
           median_minutes = ('minutes_in', lambda x: x.quantile(0.5)),
           q_75_minutes = ('minutes_in', lambda x: x.quantile(0.75)))
agg_df_q = agg_df_q[['word', 'q_25_minutes', 'median_minutes', 'q_75_minutes']]

agg_df_q

,word,q_25_minutes,median_minutes,q_75_minutes
0,ass,36.6100,76.820,116.3800
1,asses,33.0625,76.565,123.9475
2,asshead,42.8600,42.860,42.8600
3,asshole,26.5200,34.370,46.1475
4,assholes,35.4800,48.480,55.8300
5,bastard,47.2550,56.505,82.8775
6,bastards,44.1200,44.120,44.1200
7,bitch,31.9975,75.015,111.7725
8,bitches,64.3200,77.360,115.0700
9,bullshit,28.4150,54.250,85.2300
